## Simple set up

In [1]:
%load_ext autoreload
%autoreload 2

## Imports

In [2]:
import ast
import janitor
import numpy as np
import pandas as pd
import skimage.measure as measure
import shapely.geometry as geometry
import shapely.ops
import src
import matplotlib.pyplot as plt
import napari

## Load data

In [3]:
roi_files = sorted(
    src.data_dir("sample", "data_cells_tracked", "xy02", "normData").glob("*.pkl")
)

tracked_cells_df = (
    # Read pickle files into a single data frame.
    pd.DataFrame(
        dict(
            list_str_cells = [pd.read_pickle(file) for file in roi_files]
        )
    )
    # Each observation is a list of strings.
    # Convertet to long format.
    .explode(column="list_str_cells")
    # Each string is a set of variables delimited by tab.
    .list_str_cells
    .str.rstrip("\t")
    .str.split(r"\t+", expand=True) # One column per variable.
    .reset_index()
    # Indicate actual column values.
    .rename(columns = {
        "index": "frame",
        0: "id",
        1: "color",
        2: "roi_id",
        3: "track_id",
        4: "roi_polygon",
        5: "center",
        6: "axis",
        7: "GFP",
        8: "DsRed",
        9: "dead",
        10: "previous_frame",
        11: "next_frame",
        12: "mother_id"
    })
    .select_columns(["GFP", "DsRed", "dead"], invert=True)
    # Transfrom shape-strings columns into geometries.
    .assign(
        roi_polygon=lambda df: df.roi_polygon.apply(ast.literal_eval).apply(geometry.Polygon),
        center=lambda df: df.center.apply(ast.literal_eval).apply(geometry.Point),
        axis=lambda df: df.axis.apply(ast.literal_eval).apply(geometry.LineString)
    )
    # Swap coordinates since napari and ImageJ has inverted axes.
    .assign(
        roi_polygon=lambda df: df.roi_polygon.apply(
            lambda polygon: shapely.ops.transform(lambda x, y: (y, x), polygon)
        )
    )
    # Transform polygons to fit napari conditions.
    .assign(
        napari_polygon=lambda df: df.apply(
            lambda df_row: np.array([[df_row.frame, x, y] for (x, y) in df_row.roi_polygon.exterior.coords]), 
            axis=1)
    ).
    assign(
        id = lambda df: df.id.str.split(".").apply(lambda x: x[1]).astype(int) + 1,
        coords = lambda df: df.roi_polygon.apply(lambda x: np.array(x.exterior.coords))
    )
)

tracked_cells_df.head(1)

,frame,id,color,roi_id,track_id,roi_polygon,center,axis,previous_frame,next_frame,mother_id,napari_polygon,coords
0,0,1,#abcdcd,roi_f20_n0_x234_y409,[],"POLYGON ((410 238, 410 239, 411 239, 411 240, ...",POINT (233.6920667302324 410.6278959176846),LINESTRING (246.9999999999999 430.683093054870...,[],[],[],"[[0.0, 410.0, 238.0], [0.0, 410.0, 239.0], [0....","[[410.0, 238.0], [410.0, 239.0], [411.0, 239.0..."


In [4]:
from dask_image.imread import imread

In [5]:
channels_stacks = {
    path.stem: imread(path.joinpath("*.tif").as_posix())
    for path in src.data_dir("sample", "tif", "xy02").glob("*")
    if path.is_dir() and path.stem == "GFP_mask"
}
channels_stacks

{'GFP_mask': dask.array<concatenate, shape=(25, 512, 640), dtype=uint8, chunksize=(1, 512, 640), chunktype=numpy.ndarray>}

In [6]:
gfp_mask = channels_stacks["GFP_mask"].compute()
matrix_shape = gfp_mask.shape[1:]
matrix_shape

(512, 640)

In [7]:
for row in tracked_cells_df.itertuples(index=False):
    
    mask = measure.grid_points_in_poly(matrix_shape, row.coords)
    gfp_mask[row.frame, ][mask] = row.id

In [8]:
def get_cmap(n, name='hsv'):
    '''Returns a function that maps each index in 0, 1, ..., n-1 to a distinct 
    RGB color; the keyword argument name must be a standard mpl colormap name.'''
    return plt.cm.get_cmap(name, n)

In [9]:
cmap = get_cmap(tracked_cells_df.id.unique().shape[0])

In [10]:
colors = {x+1: cmap(x) for x in range(tracked_cells_df.id.unique().shape[0])}
colors = {}

In [11]:
%gui qt5
viewer = napari.Viewer()
viewer.add_labels(
        data=gfp_mask,
        color=colors,
        name="labels",
        blending="additive"
)

/Users/jvelezmagic/opt/anaconda3/envs/cellviewer/lib/python3.9/site-packages/napari/_qt/qt_main_window.py:285: RuntimeWarning: The window geometry settings could not be loaded due to the following error: cannot unpack non-iterable NoneType object
  self.show()


<Labels layer 'labels' at 0x17681e4c0>